# KAWA Administration API

Use the following snipper to authenticate in KAWA, using the credentials from the `.env` file.

Please refer to README.md to setup your environmnent.

All those commands require an `ADMIN` or a `SETUP_ADMIN` account to be executed.


In [ ]:
from kywy.client.kawa_client import KawaClient as K

kawa = K.load_client_from_environment()
cmd = kawa.commands

## 1. User management


### 1.a Create users

This snippet will create two users with the specified password and names and add them to the workspace called WAYNE.
If the workspace does not exist, it will be created.

In [ ]:
cmd.create_users_in_workspace(
    user_list=[
        {
            'email': 'bruce@wayne.com',
            'password': 'aA1!dzfegrS$S',
            'first_name': 'Bruce',
            'last_name': 'Wayne',
        },
        {
            'email': 'lucius@wayne.com',
            'password': 'hts6Hrsgs%rte',
            'first_name': 'Lucius',
            'last_name': 'Fox',
        },
    ],
    workspace_name='WAYNE',
    
    # Set this to True when your instance has a whitelist of user.
    # DO NOT use if you are unsure. It will create a white list if it does not exit and 
    # potentially cut out every user who can authenticate.
    add_users_to_whitelist=False,
)



### 1.b Change the password for a user

In [ ]:
cmd.change_user_password(
    email_or_unique_id='lucius@wayne.com', 
    new_password='bB7@klk45DSGgrs!op',
);

### 1.c Change the role of a user

The available roles in KAWA are `ADMIN_ROLE` and `READER_ROLE`.

ADMIN users will be able to manage the application user accounts. They will also be granted all the privileges in KAWA, in particular: access to all workspapces and access to all data, regardless of Row Level and Column Level security policies.

A third role is available which cannot be assigned (nor removed): `SETUP_ADMIN_ROLE`. Only one user in KAWA will have this privilege.

In [ ]:
cmd.replace_user_role(
    email_or_unique_id='bruce@wayne.com', 
    new_role='READER_ROLE',
);

### 1.d Replace the overall permissions of users

Those permissions apply to one user across the entire app, regardless of the workspace in which the user is authenticated.

The available permissions are:
- GENERATIVE_AI: Grant to the user the access to AI features on all workspaces

In [ ]:
cmd.replace_permissions(
    user_email_or_id_list=['lucius@wayne.com'],
    permissions=['GENERATIVE_AI'],
);

### 1.e Replace the forbidden data source types for users

This will restrict type of data users can upload into KAWA.
It does not affect the data that users can read from KAWA.

The available types are:
- TRANSFORMATION: Allows to create transformed datasource from sheets
- TIME_SERIES: Allows to create specific TimeSeries datasources for advanced TimeSeries Analytics
- USER_FILE: Allows to upload files into KAWA (csv, csv.gz, etc...)
- EXTERNAL_SYSTEM: Allows to set up ETLs from external systems such as a database, google sheet, etc..
- PYTHON_CLIENT: Allows to load data into KAWA via the python data loader
- LIVE_CONNECT: Allows to add tables from the underlying data warehouse


In [ ]:
cmd.replace_forbidden_data_types(
    user_email_or_id_list=[ 'lucius@wayne.com'],
    forbidden_types=['TRANSFORMATION','TIME_SERIES','LIVE_CONNECT', 'EXTERNAL_SYSTEM'],
);

### 1.f Activate and Deactivate user accounts

This allows administrator to deactivate or activate user accounts. Once a user's account has been deactivated, they immediately lose access to KAWA, GUI and Python client.

In [ ]:
cmd.toggle_users_status(
    user_email_or_id_list=[ 'lucius@wayne.com'],
    active=True,
);

## 2. Global management

### 2.a Activate Features

This will toggle the feature flags of the KAWA instance.
The available flags are:

- data-patching (Allows users to edit values within the grid views)
- tutorial-videos (Add a list of videos in the right vertical section)
- support-chat (Enable support chat with kawa.ai team)
- ai-support (Enable AI features in the app)
- list-principals (Allow to show all the users in user selection lists - like members of workspaces etc..)
- scripts (Enable script support)
- automations (Enable automation support)
- dashboard-export (Enable dashboard export)
- data-sample (Shows a list of samples in the datasource section)
- user-sign-up (When enabled, users can signup by email through the login page)


In [ ]:
cmd.toggle_feature(feature_name='data-samples', is_enabled=True)
cmd.toggle_feature(feature_name='list-principals', is_enabled=True)
cmd.toggle_feature(feature_name='automations', is_enabled=True);
cmd.toggle_feature(feature_name='ai-support', is_enabled=True);
cmd.toggle_feature(feature_name='agents', is_enabled=True);
cmd.toggle_feature(feature_name='scripts', is_enabled=True);

### 2.b Associate a license file to the KAWA instance

This allows to load a license file into a running kawa instance. License files are json files.
If you need a license, please visit www.kawa.ai.

In [ ]:
kawa.commands.set_license(path_to_license_file='/tmp/short.json')

### 2.c Activate a Python runner for a workspace

This will attach a python runner in a workspace, allowing to benefit from all the Python powered features of the platfom.
Please refer to https://docs.kawa.ai/python-script-runners#167e42f9ebbe42518f1e9655c6a8853c for detils about the Python runner configuration.

In [ ]:
import os
import json

# Will activate a runner in this workspace
WORKSPACE_ID=4
kawa.set_active_workspace_id(workspace_id=WORKSPACE_ID)

cmd.add_script_runner(
    name='Main runner',
    host='runner.wayne.com',
    port=8815,
    tls=False,
    private_key='2ef105d2a8c90074cfe1711cf7cb79906b94f2436ceba3198a4f8ce0e11ed825'
)

health = kawa.runner_health()

# Check the "healthy" flag, it should be set to True
print(json.dumps(health, indent=4))

### 2.d Modify a script runner configuration

The command to create a runner, if executed multiple times with different parameters, will not amend the configuration of existing runners. Please use the following commands to do so.

In [ ]:
# Will modify a runner in this workspace
WORKSPACE_ID=2
kawa.set_active_workspace_id(workspace_id=WORKSPACE_ID)

cmd.replace_script_runner_location(host='127.0.0.1', port=8816, tls=False)
cmd.replace_script_runner_private_key(private_key=os.getenv('KAWA_AUTOMATION_SERVER_AES_KEY'));

### 2.e Define and configure a Global Python runner

By enabling a Global Runner, all the workspaces will automatically inherit from this configuration.
It can be overriden useing the commands described above.
A restart of KAWA is necessary for this to be taken in account.

In [ ]:
cmd.replace_configuration('GlobalPythonRunnerConfig', {
    'name':'Main runner',
    'host':'kawa-script-runner',
    'port':8815,
    'tls':False,
    'enabled':True
})

### 2.f Manage python dependencies (When not using a third party Version Control System)

When using the KAWA platform to store your scripts (this is defined on the GUI per workspace), you can set the list of available modules using the following command. The syntax is similar to the one used in a standard `requirements.txt` file.

By default, the following packages are installed:

```
pandas>=2.2.2
faker>=28.4.1
yfinance>=0.2.41
slack_sdk==3.31.0
tabulate==0.9.0
gender-detector==0.1.0
textblob==0.18.0
scipy==1.14.0
scikit-learn==1.5.1
geopy==2.4.1
```


(Note that nothing needs to be restarted for this change to be taken in account)


In [ ]:
cmd.replace_configuration('LocalFileStoreConfig', {
    'requirements': [
        "pandas>=2.2.2",
        "faker>=37.0.0",
        "kywy>=0.29.2",
        "google-cloud-pubsub==2.29.0",
        "yfinance>=0.2.41",
        "slack_sdk==3.31.0",
        "tabulate==0.9.0",
        "gender-detector==0.1.0",
        "textblob==0.18.0",
        "scipy==1.14.0",
        "scikit-learn==1.5.1",
        "geopy==2.4.1",
        "psycopg2-binary",
        "gspread",
        "oauth2client",
        "xgboost",
        "boto3==1.26.112"
  ]
})

### 2.g Manage workspaces

This step lets you choose the behavious of workspaces creation.

__Please note that the server needs to be restarted for these parameters to be taken in account.__

In [ ]:
cmd.replace_configuration('WorkspaceConfiguration', {
    
    # If set to True, then only admins will be able to create new workspaces
    'workspaceCreationRestrictedToAdmins': True,
    
    # Two possible values: 
    # 'CREATE_NEW'  > KAWA will create a personal workspace for the user when they login for the first time
    # 'USE_DEFAULT' > KAWA will add all new users into 'defaultWorkspaceId'
    'workspaceInitializationMode': 'USE_DEFAULT', 
    
    # This is required if the workspaceInitializationMode is set to USE_DEFAULT
    'defaultWorkspaceId': '1', 
})

ℹ️ If you whish to archive a workspace, use the following command.

The second argument: `migrateMembersToWorkspaceId` will migrate users whose home workspace is the one being archived. The home workspace is the workspace to which users will be redirected to if they lose access to their current workspace.

In [ ]:
cmd.run_command(
   command_name='ArchiveWorkspace',
   command_parameters={
           'workspaceId':'Workspace id to delete',
           'migrateMembersToWorkspaceId':'Migrate users over to this workspace id'
   }
)

## 3. Reporting

### 3.a User and workspace members reporting

In [ ]:
# List of all users in KAWA
kawa.reporting().generate_user_list_report()

In [ ]:
# List of workspace members in KAWA
kawa.reporting().generate_workspace_member_list_report()

## 4. Query cache operations

The query cache can be invalidated from the kywy library. 
If your setup relies on Live Connections _(to an external warehouse/lakehouse or to databases that are not managed by KAWA)_, use the following command from your ETL processes to invalidate the query caches at the appropriate times.

> ⚠️ This command will return the list of the KAWA datasources whose query caches were invalidated. If it returns an empty array, it means that no datasources were found on KAWA associated with the table. You might consider double checking the paramters of the command.

In [ ]:
cmd.run_command(
    command_name='ClearQueryCacheForTable',
    command_parameters={
        'catalog':'default', # Name of your database/catalog
        'schema': '',        # Name of the schema > It is an empty string for Clickhouse
        'table': 'trades'    # Name of the table
    }
)